In [1]:
import simpy
import random
import numpy as np
from simpy.events import Event

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

SHIFT = 480

ANSWERED_CALLS = 1000
FINISHED_T = 0
all_customers = []
operation_t = [0,0]
shift_number = 0

In [3]:
def service(env, opr):
    if opr==operator1:
        time = random.lognormvariate(M,S)
        operation_t[0] += time
        yield env.timeout(time)
        
    elif opr==operator2:
        time = random.uniform(*service_range)
        operation_t[1] += time
        yield env.timeout(time)

In [4]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.waiting_t = 0
        
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
        
        
        # Voice recognition and routing
        if router.is_idle():
            router.working_t += [self.env.now]    
        router.count += [self.name]
        
        yield self.env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router.count.remove(self.name)
        if router.is_idle():
            router.idle_t += [self.env.now]
        
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            all_customers.append(self)
            self.operator = None
            return 
        
        print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request(0) as req:
            self.waiting_t = self.env.now
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
                print('%s is left the operator %d after 10 minutes at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
                all_customers.append(self)
                self.operator = None
                return 
            self.waiting_t = self.env.now - self.waiting_t
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            yield self.env.process(service(env,self.operator))
            print('%s is done with operator %d at %g' % (self.name,
                                                        (self.operator==operator2)+1,
                                                        self.env.now))
            all_customers.append(self)

In [5]:
class robocall:
    def __init__(self,env):
        self.count = []
        self.working_t = []
        self.idle_t = [0]
        self.env = env
        
    def answer(self):
        return len(self.count) < 100
    
    def is_idle(self):
        return len(self.count) == 0
            
    def utilization(self):
        return sum([ w-i for i,w in zip(router.idle_t, router.working_t)]) / FINISHED_T

In [6]:
def breaks():
    return [random.uniform(0,SHIFT) for i in  range(np.random.poisson(8))]

In [7]:
def shift(env):
    global shift_number
    shift_number += 1
    
    [env.process(take_a_break(env, b, operator1)) for b in breaks()]
    [env.process(take_a_break(env, b, operator2)) for b in breaks()]

    yield env.timeout(SHIFT)
    print("the time %g" % env.now)

    env.process(shift(env))

In [8]:
def take_a_break(env, number, opr):
    yield env.timeout(number)
    with opr.request(1) as brk:
        yield brk
        
        if SHIFT*shift_number - env.now > 0:
            if SHIFT*shift_number - env.now >= 3:
                print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(3)
            else:
                print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(480*shift_number - env.now)
            print("break ended for operator %d at %g" % ((opr == operator2)+1 , env.now))

In [9]:
def customer_generator(env):
    start = True
    
    num_answered = 0;
    
    while(len(all_customers)<ANSWERED_CALLS):
        
        #Every 480 a new shift starts.
        if start:
            env.process(shift(env))
            start = False
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if router.answer():
            num_answered += 1
            customer = Customer('Customer %s' %(num_answered), env)
            
    FINISHED_T = env.now 
    finished.succeed()

In [10]:
env = simpy.Environment()

finished = Event(env)
router = robocall(env)

operator1 = simpy.PriorityResource(env, capacity = 1)
operator2 = simpy.PriorityResource(env, capacity = 1)

env.process(customer_generator(env))
env.run(finished) 
FINISHED_T = env.now

Customer 1 initiated a call at 26.5136
break started for operator 2 at 34.076
Customer 2 initiated a call at 34.9192
Customer 1 is rooted to operator 1 at 35.9632
Customer 1 is assigned to the operator 1 at 35.9632
break ended for operator 2 at 37.076
Customer 2 is rooted to operator 2 at 37.3759
Customer 2 is assigned to the operator 2 at 37.3759
Customer 2 is done with operator 2 at 43.0204
Customer 1 is done with operator 1 at 46.119
break started for operator 1 at 46.119
Customer 3 initiated a call at 46.5973
Customer 3 is rooted to operator 2 at 46.7038
Customer 3 is assigned to the operator 2 at 46.7038
break ended for operator 1 at 49.119
Customer 4 initiated a call at 51.3862
Customer 3 is done with operator 2 at 52.0645
Customer 4 hangs up the call at 61.3177
Customer 5 initiated a call at 61.9361
Customer 5 is rooted to operator 2 at 67.0397
Customer 5 is assigned to the operator 2 at 67.0397
Customer 6 initiated a call at 67.1342
Customer 7 initiated a call at 68.5859
Custom

Customer 465 is done with operator 2 at 2836.21
Customer 464 is assigned to the operator 2 at 2836.21
Customer 459 is done with operator 1 at 2838.01
Customer 467 is assigned to the operator 1 at 2838.01
Customer 464 is done with operator 2 at 2838.17
break started for operator 2 at 2838.17
Customer 466 hangs up the call at 2838.41
break ended for operator 2 at 2841.17
break started for operator 2 at 2841.17
Customer 468 initiated a call at 2843.91
break ended for operator 2 at 2844.17
Customer 467 is done with operator 1 at 2850.49
Customer 469 initiated a call at 2858.48
break started for operator 1 at 2859.38
Customer 469 is rooted to operator 2 at 2860.74
Customer 469 is assigned to the operator 2 at 2860.74
break ended for operator 1 at 2862.38
Customer 469 is done with operator 2 at 2863.19
Customer 470 initiated a call at 2864.8
Customer 468 is rooted to operator 2 at 2866.41
Customer 468 is assigned to the operator 2 at 2866.41
break started for operator 1 at 2868.46
Customer 4

Customer 921 is assigned to the operator 2 at 5907.16
Customer 921 is done with operator 2 at 5910.42
Customer 923 is assigned to the operator 2 at 5910.42
Customer 923 is done with operator 2 at 5916.34
break started for operator 2 at 5916.34
Customer 922 is done with operator 1 at 5916.7
break ended for operator 2 at 5919.34
Customer 924 initiated a call at 5922.39
Customer 924 is rooted to operator 2 at 5922.39
Customer 924 is assigned to the operator 2 at 5922.39
Customer 925 initiated a call at 5924.29
Customer 924 is done with operator 2 at 5929.36
break started for operator 2 at 5929.36
break ended for operator 2 at 5932.36
Customer 926 initiated a call at 5932.48
Customer 925 is rooted to operator 2 at 5933.92
Customer 925 is assigned to the operator 2 at 5933.92
Customer 927 initiated a call at 5934.82
Customer 928 initiated a call at 5935.81
Customer 925 is done with operator 2 at 5937.44
Customer 927 is rooted to operator 1 at 5939.07
Customer 927 is assigned to the operator

In [11]:
print('Voice Recognition System Utilization: ', router.utilization())

Voice Recognition System Utilization:  0.46815216487473793


In [12]:
print('Utilization Operator 1:', operation_t[0]/FINISHED_T, 
      '\nUtilization Operator 2:' ,operation_t[1]/FINISHED_T)

Utilization Operator 1: 0.4171894106190104 
Utilization Operator 2: 0.3902070630854685


In [13]:
customer_waiting = [c.waiting_t for c in all_customers if c.operator != None]
print('Average Waiting Time: ', sum(customer_waiting)/len(customer_waiting))

Average Waiting Time:  1.5244526424035285
